<a href="https://colab.research.google.com/github/csepulveda7/SpotApp/blob/main/injury_risk_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import all necessary libraries and tools
import pandas as pd
import datetime
import psycopg2
from datetime import date
import numpy as np

# Connecting to the database
connection = psycopg2.connect(user="panjfsdgbbpqjp",
                          password="e82392fdcc0e809c9ee02a9459d9da3e0e414094bdbc8f88acb69d3f7686da5a",
                          host="ec2-52-44-31-100.compute-1.amazonaws.com",
                          port="5432",
                          database="d2itj4h57p7v0h")

# Retreive all the readiness data from ff90
cursor = connection.cursor()
schema = "\"UCF_Women_Soccer_DEV\""

cursor.execute('SELECT * FROM {}.ff90_training_load'.format(schema)) 
training_load =  pd.DataFrame(cursor.fetchall())
cursor.execute('SELECT * FROM {}.ff90_soreness'.format(schema)) 
soreness =  pd.DataFrame(cursor.fetchall())
cursor.execute('SELECT * FROM {}.ff90_readiness'.format(schema)) 
readiness =  pd.DataFrame(cursor.fetchall())
cursor.execute('SELECT * FROM {}.ff90_hydration'.format(schema)) 
hydration = pd.DataFrame(cursor.fetchall())
cursor.execute('SELECT * FROM {}.ff90_notes'.format(schema)) 
notes =  pd.DataFrame(cursor.fetchall())

# Retreive all the data from Polar
cursor.execute('SELECT * FROM {}.polar_session'.format(schema)) 
polar_data = pd.DataFrame(cursor.fetchall())
polar_col_names = [desc[0] for desc in cursor.description]

# Retreive all the data from InStat
cursor.execute('SELECT * FROM {}.instatdata'.format(schema))    
instat_data  = pd.DataFrame(cursor.fetchall())
instat_col_names = [desc[0] for desc in cursor.description]

# Replace column numbers with names
training_load.rename(columns = {0:'Date', 1:'Time', 2:'Last', 3:'First', 4:'Rating', 5:'Duration(min)', 6:'Load', 7:'Category', 8:'Team'}, inplace = True)
soreness.rename(columns = {0:'Date', 1: 'Time', 2: 'Last', 3: 'First', 4: 'Muscle Group', 5: 'Soreness level', 6: 'Soreness Label'}, inplace = True)
readiness.rename(columns = {0:'Date', 1:'Time', 2:'Last', 3:'First', 4:'Score', 5:'Fatigue', 6:'Mood', 7:'Soreness', 8:'Stress', 9:'Sleep Quality', 10:'Sleep Hours'}, inplace = True)
hydration.rename(columns = {0:'Date', 1: 'Time', 2: 'Last', 3: 'First', 4: 'Hydration Level', 5: 'Hydration Label'}, inplace = True)
notes.rename(columns = {0:'Date', 1:'Time',2: 'Last', 3:'First', 4:'Category',5:'Content'}, inplace = True)
polar_data.columns = polar_col_names
instat_data.columns = instat_col_names

Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [18]:
(training_load.head(3).style)

,Date,Time,Last,First,Rating,Duration(min),Load,Category,Team
0,2020-08-07,15:46:00,Agin,Emma,3,19,57,Recovery,Individual
1,2020-08-07,15:30:00,Anderson,Brooke,9,60,540,Team Training,UCF Knights
2,2020-08-07,15:47:00,Barkett,Jade,2,19,38,Recovery,Individual


In [19]:
(training_load.shape)

(22885, 9)

In [4]:
readiness.head(3)

,Date,Time,Last,First,Score,Fatigue,Mood,Soreness,Stress,Sleep Quality,Sleep Hours
0,2020-08-07,12:34:00,Agin,Emma,94.00,3,3,0,3,3,9.0
1,2020-08-07,12:10:00,Anderson,Brooke,95.75,2,2,2,2,2,8.0
2,2020-08-07,12:54:00,Barkett,Jade,69.34,-1,1,-1,1,1,7.0


In [20]:
readiness.shape

(23939, 11)

In [27]:
readiness['Date'].unique().shape

(1948,)

In [5]:
hydration.head(3)

,Date,Time,Last,First,Hydration Level,Hydration Label
0,2020-07-26,02:49:00,Agin,Emma,1,Hyper Hydrated
1,2020-07-25,03:42:00,Agin,Emma,1,Hyper Hydrated
2,2020-07-23,14:19:00,Agin,Emma,2,Well Hydrated


In [21]:
hydration.shape

(1256, 6)

In [26]:
hydration['Date'].unique().shape

(927,)

In [24]:
import plotly.express as px
fig = px.box(hydration, y="Hydration Level")
fig.show()

In [6]:
notes.head(3)

,Date,Time,Last,First,Category,Content
0,2020-06-26,02:46:00,Lewis,Olivia,general,Wisdom Teeth Recovery
1,2020-06-24,19:17:00,Lewis,Olivia,general,Wisdom Teeth Recovery
2,2020-06-24,17:59:00,Olsson,Mallory,general,Sore/overstrained right glute. Been resting fo...


In [7]:
polar_data.head(3)

,player_number,player_name,session_name,session_type,phase_name,duration,start_time,end_time,hr_min,hr_avg,...,num_acc_n1p99_n1,num_acc_np99_np5,num_acc_p5_p99,num_acc_1_1p99,num_acc_2_2p99,num_acc_3_50,calories,training_load_score,recovery_time,cardio_load
0,18,Jade Barkett,None,Training,Whole session,02:05:27,13-02-2020 13:43:29,13-02-2020 15:48:56,66,131,...,225,473,365,229,36,7,778,113,15.1,NaN
1,24,Emma Agin,NaN,Training,Whole session,01:31:45,05-02-2021 14:28:14,05-02-2021 15:59:59,113,166,...,256,345,369,196,68,17,626,117,10.4,NaN
2,18,Jade Barkett,NaN,Training,Whole session,01:31:45,05-02-2021 14:28:14,05-02-2021 15:59:59,88,141,...,245,415,319,248,76,10,600,89,9.0,NaN


In [28]:
polar_data.shape

(28608, 41)

In [53]:
num_whole = 0
for a in polar_data['phase_name']:
  if (a == 'Whole session'):
    num_whole = num_whole + 1

print(num_whole)

10495


In [57]:
polar_data['session_type'].value_counts()

Training                     26144
Game                          1676
Match                          324
Strength and conditioning      300
Other                          149
Test                            15
Name: session_type, dtype: int64